In [1]:
import pandas as pd
from langchain.document_loaders.csv_loader import CSVLoader

pd.read_csv("feed_to_rag.csv", dtype=str).to_csv("test_rag.csv", index=False)

In [2]:
loader = CSVLoader(
    file_path="test_rag.csv",
    source_column="Code",
    csv_args={
        "delimiter": ",",
        "quotechar": '"',
    },
)
data = loader.load()

In [3]:
import os

from langchain.chains.query_constructor.base import AttributeInfo
from langchain.embeddings import OllamaEmbeddings
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.vectorstores import Chroma

use_ollama_embeddings = False

embedding_function = (
    SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")
    if not use_ollama_embeddings
    else OllamaEmbeddings(
        base_url="http://192.168.28.4:11434",
        model="mistral",
    )
)


def get_embedding_function():
    if not hasattr(get_embedding_function, "embedding_fn"):
        get_embedding_function.embedding_fn = embedding_function
    return get_embedding_function.embedding_fn


if not os.path.exists("embeddings.db"):
    db = Chroma.from_documents(
        data, get_embedding_function(), persist_directory="./embeddings.db"
    )
    db.persist()
    db = None
vectorstore = Chroma(
    persist_directory="./embeddings.db", embedding_function=get_embedding_function()
)


ollama_endpoint = "http://192.168.28.4:11434"


# Create and load LLM
def return_ollama_llm(base_url=ollama_endpoint, repeat_penalty=1.4, temperature=0.0):
    from langchain.callbacks.manager import CallbackManager
    from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
    from langchain.llms import Ollama

    if not hasattr(return_ollama_llm, "ollama"):
        return_ollama_llm.ollama = Ollama(
            model="mistral",
            base_url=ollama_endpoint,
            repeat_penalty=repeat_penalty,
            temperature=temperature,
            callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
        )
    return return_ollama_llm.ollama


llm = return_ollama_llm()

## Rewrite-Retrieve-Read Implementation

In [284]:
import re

from langchain.chains import RetrievalQAWithSourcesChain, VectorDBQA
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough

answer_pattern = r"\+\+(.*?)\+\+|\+\+(.*?)$"
correcter_template = """You are a customs officer. Based on a short cargo label,
add a few descriptive tags (10 words or less) that best detail the cargo if the label is even marginally ambiguous for tariff coding, else just provide a cleaned label.
Be extremely brief.
Label:
{query}
Answer:
"""

coder_template = """Based on the cargo description, present 3 best 8 digit tariff codes, in order of relevance. Be extremely brief and succinct.
Present numeric codes ONLY -- and a short description per code in 10 words or less. 
Be brief and direct as [{{"code":"xxxxxxxx", "summary":"summary 1"}}, {{"code":"xxxxxxxx", "summary":"summary 2"}}, {{"code":"xxxxxxxx", "summary":"summary 3"}}]
Description:
{context}
"""


rewrite_prompt = ChatPromptTemplate.from_template(correcter_template)
coder_prompt = ChatPromptTemplate.from_template(coder_template)


def _parse(text):
    # Use re.findall to extract all occurrences of text between "++"
    if "++" in text:
        try:
            matches = re.findall(answer_pattern, text)
            extracted_text = matches[0][0] if matches[0][0] else matches[0][1]
            return extracted_text.strip()
        except:
            return text.strip()
    elif """"tag":""" in text:
        try:
            return json.loads(text)['tag']
        except:
            return text.strip()
    else:
        return text.strip()


rewriter_chain = RunnablePassthrough.assign(query=lambda x: x) | {
    "answer": (rewrite_prompt | llm | _parse),
    "original": lambda x: x["query"]["query"],
}
code_chain = VectorDBQA.from_llm(
    llm, prompt=coder_prompt, vectorstore=vectorstore, search_type="mmr", k=3
)
from langchain.schema.runnable import RunnablePassthrough

chain = (
    {"query": RunnablePassthrough()}
    | rewriter_chain
    | {"query": lambda x: f""""{x["original"]["query"]}", "{x["answer"]}" """}
    | code_chain
    | {"response": lambda x: x["result"]}
)

/opt/miniconda3/lib/python3.11/site-packages/langchain/chains/retrieval_qa/base.py:251: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


In [260]:
rewriter_chain.invoke({"query": "tv"})

{"tag": "electronics", "subcategory": "television"}

{'answer': '{"tag": "electronics", "subcategory": "television"}',
 'original': 'tv'}

In [261]:
code_chain({"query": "chicken"})

1. Code: 01050000 - Live poultry (fowl) weighing not more than 185g, including guinea fowls.
Description: Gallus domesticus live birds up to 185 grams in weight, which includes guineas. This code is relevant for the context of transporting and importing these specific types of poultry.

{'query': 'chicken',
 'result': '1. Code: 01050000 - Live poultry (fowl) weighing not more than 185g, including guinea fowls.\nDescription: Gallus domesticus live birds up to 185 grams in weight, which includes guineas. This code is relevant for the context of transporting and importing these specific types of poultry.'}

In [262]:
chain.invoke({"query": "chicken and yogurt"})

Chick Yoghurt (Food)1. Code: 84593700 - Eggs, uncooked (shell)
Description: containing eggs
Hierarchic Description: shell eggs not cooked or otherwise prepared such as boiled fried scrambled >> egg products
Level: 2
2. Code: 16018459 - Jams and jellies of fruit, nut pastes obtained by cooking (homogenised)
Description: homogenized preparations other
Hierarchic Description: jam or jelly made from fruits or nuts cooked to a thick consistency >> spreads
Level: 2
3. Code: 16049857 - Other vegetables and mixtures of vegetables (homogenised)
Description: other vegetables and mixtures of vegetables
Hierarchic Description: vegetable preparation not made by vinegar or acetic acid, frozen >> mixed vegetables

{'response': '1. Code: 84593700 - Eggs, uncooked (shell)\nDescription: containing eggs\nHierarchic Description: shell eggs not cooked or otherwise prepared such as boiled fried scrambled >> egg products\nLevel: 2\n2. Code: 16018459 - Jams and jellies of fruit, nut pastes obtained by cooking (homogenised)\nDescription: homogenized preparations other\nHierarchic Description: jam or jelly made from fruits or nuts cooked to a thick consistency >> spreads\nLevel: 2\n3. Code: 16049857 - Other vegetables and mixtures of vegetables (homogenised)\nDescription: other vegetables and mixtures of vegetables\nHierarchic Description: vegetable preparation not made by vinegar or acetic acid, frozen >> mixed vegetables'}

In [263]:
chain.invoke({"query": "halaal"})

Halaal food productsCode: 04100000 - Animal products (not elsewhere specified)
Description: Edible animal origin goods.

{'response': 'Code: 04100000 - Animal products (not elsewhere specified)\nDescription: Edible animal origin goods.'}

In [264]:
chain.invoke({"query": "drones for farming"})

Drones used in agriculture1. Code: 84368000 - machinery excluding germination plant fitted with equip (other agricultural, horticultural or forestry equipment)
2. Code: 23080000 - vegetable materials and waste in pellets used for animal feeding (not elsewhere specified or included)
3. Code: 05119999 - other section II vegetable products, agglomerated by compression or binder up to 3% weight chapter 6 live trees and plants cut flowers etc., subject to second part of heading

{'response': '1. Code: 84368000 - machinery excluding germination plant fitted with equip (other agricultural, horticultural or forestry equipment)\n2. Code: 23080000 - vegetable materials and waste in pellets used for animal feeding (not elsewhere specified or included)\n3. Code: 05119999 - other section II vegetable products, agglomerated by compression or binder up to 3% weight chapter 6 live trees and plants cut flowers etc., subject to second part of heading'}

In [265]:
chain.invoke({"query": "solar cells"})

"Solar panels", "Photovoltaic equipment".1. Code: 8547092 - Solar Energy Apparatus (Other)
Description: Other machines, apparatus for electrical machines other than those specified in this chapter or elsewhere in the tariff code hierarchy. This includes equipment and gadgets based on solar energy that do not fit into any of the more specific categories listed herein.
2. Code: 85417096 - Solar Energy Photovoltaic Cells (Assembled)
Description: Assemblies or panels made up from photovoltaic cells, which are used to convert sunlight directly into electricity through a process called the photoelectric effect. These assemblies can be mounted on rooftops, walls, or other surfaces and connected to an inverter for use with household appliances or grid-tied systems.
3. Code: 7615902 - Solar Energy Collectors (Other)
Description: Other types of solar collectors that do not fit into any of the more specific categories listed herein, such as flat plate collectors, evacuated tube collectors, or con

{'response': '1. Code: 8547092 - Solar Energy Apparatus (Other)\nDescription: Other machines, apparatus for electrical machines other than those specified in this chapter or elsewhere in the tariff code hierarchy. This includes equipment and gadgets based on solar energy that do not fit into any of the more specific categories listed herein.\n2. Code: 85417096 - Solar Energy Photovoltaic Cells (Assembled)\nDescription: Assemblies or panels made up from photovoltaic cells, which are used to convert sunlight directly into electricity through a process called the photoelectric effect. These assemblies can be mounted on rooftops, walls, or other surfaces and connected to an inverter for use with household appliances or grid-tied systems.\n3. Code: 7615902 - Solar Energy Collectors (Other)\nDescription: Other types of solar collectors that do not fit into any of the more specific categories listed herein, such as flat plate collectors, evacuated tube collectors, or concentrating collector s

In [266]:
chain.invoke({"query": 'idols'})

"Artifacts", "Religious items", "Sculptures".{
"code": "70189010",
"summary": "glass statues"
},
{"code":"69130000","summary":"statuettes and ornamental ceramic articles"},
{"code":"97039000","summary":"other original sculptures in any material"}

{'response': '{\n"code": "70189010",\n"summary": "glass statues"\n},\n{"code":"69130000","summary":"statuettes and ornamental ceramic articles"},\n{"code":"97039000","summary":"other original sculptures in any material"}'}

In [267]:
chain.invoke({"query": 'ww2 raf helmet gift'})

{"tag": "WWII RAF Helmet Gift"}1. Code: 65061010 - Speed glass welding helmets and industrial use helmets (HierarchicDescription: Other safety headwear)
2. Code: 65059000 - Lace textile fabric hats, hair nets, etc. (HierarchicDescription: Hats made from laced or crocheted fabrics in pieces but not strips)

{'response': '1. Code: 65061010 - Speed glass welding helmets and industrial use helmets (HierarchicDescription: Other safety headwear)\n2. Code: 65059000 - Lace textile fabric hats, hair nets, etc. (HierarchicDescription: Hats made from laced or crocheted fabrics in pieces but not strips)'}

In [273]:
chain.invoke({"query": 'Bluray DVD'})

"DVD", "Media",1. Code: 85243920, Summary: Other Video Compact Discs (Incorporating screens without drivers or control circuits)
2. Code: 85238040, Summary: Children's Video Films (Disks, Tapes, Storage Devices for Recording of Phenomena Recorded Including Matrices and Masters for Production of Discs but Excluding Products of Chapter 37 Magnetic Media)
3. Code: 85234160, Summary: DVDs

{'response': "1. Code: 85243920, Summary: Other Video Compact Discs (Incorporating screens without drivers or control circuits)\n2. Code: 85238040, Summary: Children's Video Films (Disks, Tapes, Storage Devices for Recording of Phenomena Recorded Including Matrices and Masters for Production of Discs but Excluding Products of Chapter 37 Magnetic Media)\n3. Code: 85234160, Summary: DVDs"}

In [295]:
import json

import ipywidgets as widgets
from IPython.display import clear_output, display


# Function to simulate the chain.invoke function
def chain_invoke(query):
    result_data = chain.invoke({"query": query})
    return result_data


# Function to format the result data into a table
def format_result_table(result_data):
    table_rows = []
    # Handle JSON data within the text
    json_data = re.findall(r"{.*?}", result_data["response"], re.DOTALL)
    for item in json_data:
        try:
            itemd = json.loads(item)
            table_rows.append((itemd.get("code"), itemd.get("summary")))
        except json.JSONDecodeError:
            pass

    # Regular expression pattern to find code and description
    pattern = r"(\d{8})\s+-\s+(.*?)\n"
    matches = re.findall(pattern, result_data["response"])
    # Iterate through the matches
    for code, description in matches:
        table_rows.append((code, description))
    return table_rows


# Create a text input widget for the query
query_input = widgets.Text(placeholder="Enter your query here", description="Query:")

# Create a button widget to trigger the function
button = widgets.Button(description="Search")

# Create a table widget to display the results
table = widgets.Output()


# Function to handle button click event
def on_button_click(b):
    # Clear the previous results
    table.clear_output()

    # Get the query from the input widget
    query = query_input.value

    # Invoke the function with the query
    result_data = chain_invoke(query)

    # Format the result into a table
    table_rows = format_result_table(result_data)

    # Display the result as a table
    with table:
        display(
            widgets.HTML(
                "<table><tr><th>Code</th><th>Description</th></tr>{}</table>".format(
                    "".join(
                        "<tr><td>{}</td><td>{}</td></tr>".format(code, description)
                        for code, description in table_rows
                    )
                )
            )
        )


# Attach the button click event handler
button.on_click(on_button_click)

# Display the input, button, and result table
display(query_input, button, table)

Text(value='', description='Query:', placeholder='Enter your query here')

Button(description='Search', style=ButtonStyle())

Output()